In [141]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
jerkOffData=pd.read_csv("sketch_200905a/JerkOff.csv")
WalkData=pd.read_csv("sketch_200905a/WalkAround.csv")
XucCatData=pd.read_csv("sketch_200905a/XucCat.csv")
HipHopData=pd.read_csv("sketch_200905a/HipHop.csv")
SitData=pd.read_csv("sketch_200905a/SitAndUsingComputer.csv")

In [3]:
jerkOffData

,id,TimeStamp,LandmarkID,AcX,AcY,AcZ,GyX,GyY,GyZ
0,0,2020-9-6-1-22-58-4168,1,6568,-13808,-10224,-3408,-686,-452
1,1,2020-9-6-1-22-58-4172,2,2776,-15804,-792,-3456,976,-8251
2,2,2020-9-6-1-22-58-4243,4,304,-16524,-5800,-3712,-1456,2218
3,3,2020-9-6-1-22-58-4331,3,-3256,-15872,1172,-4064,324,-453
4,4,2020-9-6-1-22-58-4644,1,-3144,-17960,-12304,-3408,14325,-19773
...,...,...,...,...,...,...,...,...,...
1083,1083,2020-9-6-1-25-13-139869,3,-4816,-12632,7784,-3920,-241,7
1084,1084,2020-9-6-1-25-14-140188,1,16588,3792,-4620,-3280,-601,147
1085,1085,2020-9-6-1-25-14-140202,2,-7528,-13376,-5940,-3488,-406,6
1086,1086,2020-9-6-1-25-14-140275,4,6912,1116,15384,-3840,-292,-72


In [6]:
jerkOffData['TimeStamp'][0]

'2020-9-6-1-22-58-4168'

In [9]:
datetime_object = datetime.strptime(jerkOffData['TimeStamp'][0], '%Y-%m-%d-%H-%M-%S-%f')

In [10]:
datetime_object

datetime.datetime(2020, 9, 6, 1, 22, 58, 416800)

In [100]:
dfjerkOff = jerkOffData[['LandmarkID','AcX','AcY','AcZ','GyX','GyY','GyZ']]
dfWalk = WalkData[['LandmarkID','AcX','AcY','AcZ','GyX','GyY','GyZ']]
dfXucCat = XucCatData[['LandmarkID','AcX','AcY','AcZ','GyX','GyY','GyZ']]
dfHipHop = HipHopData[['LandmarkID','AcX','AcY','AcZ','GyX','GyY','GyZ']]
dfSit = SitData[['LandmarkID','AcX','AcY','AcZ','GyX','GyY','GyZ']]

In [126]:
def flatten_normalize(df1,types="default",label=-1):
    newdf = pd.DataFrame( columns =['AcX_1','AcY_1','AcZ_1','GyX_1','GyY_1','GyZ_1','AcX_2','AcY_2','AcZ_2','GyX_2','GyY_2','GyZ_2','AcX_3','AcY_3','AcZ_3','GyX_3','GyY_3','GyZ_3','AcX_4','AcY_4','AcZ_4','GyX_4','GyY_4','GyZ_4']) 

    acceptPack=[1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4]
    madik={}
    for index, row in df1.iterrows():
        for keys in (dict(row)):
            if(keys!='LandmarkID'):
                if(row['LandmarkID'] in acceptPack):
                    acceptPack.remove(row['LandmarkID'])
                    #print(str(keys)+"_"+str(row['LandmarkID']))
                    madik[str(keys)+"_"+str(row['LandmarkID'])]=row[str(keys)]  
        if(len(acceptPack)<1):
            newdf=newdf.append(madik,ignore_index=True)
            madik={}
            acceptPack=[1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4]
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(newdf.values)
    
    if types=="numpy" and label!=-1:
        x_fn=pd.DataFrame(x_scaled).values
        y_fn=np.zeros((x_fn.shape[0], 1))
        y_fn.fill(label)
        return x_fn,y_fn
    else:   
        return pd.DataFrame(x_scaled)

In [138]:
x_1,y_1 = flatten_normalize(dfjerkOff,"numpy",1)
x_2,y_2 = flatten_normalize(dfWalk,"numpy",2)
x_3,y_3 = flatten_normalize(dfXucCat,"numpy",3)
x_4,y_4 = flatten_normalize(dfHipHop,"numpy",4)
x_5,y_5 = flatten_normalize(dfSit,"numpy",5)

In [139]:
X=np.concatenate((x_1, x_2,x_3,x_4,x_5))
Y=np.concatenate((y_1, y_2,y_3,y_4,y_5))

In [146]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

In [149]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [152]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 24

units = 64
output_size = 6  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [153]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=1
)

Train on 886 samples, validate on 437 samples
886/886 [==============================] - 2s 2ms/sample - loss: 1.7858 - acc: 0.2720 - val_loss: 1.7719 - val_acc: 0.3776
